# Trader Simulation in Mesa

I will attempt to build an identical model to mine in the Mesa framework. 

## Collision simulation
First I will attempt to build the collision simulation

### Initialising packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as constant
from scipy.optimize import curve_fit
import random
import math
import scipy.stats as stats
from tqdm.notebook import tqdm, trange
import ipywidgets as widgets
import mesa
import pandas as pd

### Writing new agent

In [6]:
class Trader(mesa.Agent):
    def __init__(self, unique_id, model, velocities = [1,1], m = 1, dimensions = 2):
        
        super().__init__(unique_id, model)
        self.dimensions = dimensions
        self._mass = m
        self._velocity = np.ones(self.dimensions)
        assert len(velocities) <= dimensions, 'Too few dimensions for provided input velocity array'
        
        for i, value in enumerate(velocities):
            self._velocity[i] = float(value)
            
        self._momentum = self._mass*self._velocity
        self.energy = 0.5*self._mass*np.linalg.norm(self._velocity)**2
        
        # Error handling
        assert dimensions > 0 and type(dimensions) == int, 'Must have positive integer dimensions'
        assert type(m) == int or type(m) == float, 'Mass must be numeric'
    
    @property
    def net_assets(self):
        return self._velocity[0] + self.commodity_value_intrinsic*self.commodities
    
    @property
    def mass(self):
        return self._mass

    @mass.setter
    def mass(self, value):
        self._mass = value
        self._momentum = self._mass*self._velocity
        self.energy = 0.5*self._mass*np.linalg.norm(self._velocity)**2
    
    @property
    def velocity(self):
        return self._velocity
    
    @velocity.setter
    def velocity(self, value):
        # print("velocity setter method called")
        self._velocity = value
        self._momentum = self._mass*self._velocity
        self.energy = 0.5*self._mass*np.linalg.norm(self._velocity)**2

    @property
    def momentum(self):
        return self._momentum
    
    @momentum.setter
    def momentum(self, value):
        # print("setter method called")
        self._momentum = value
        self._velocity = self._momentum/self.mass
        self.energy = 0.5*self._mass*np.linalg.norm(self._velocity)**2
        
    @property
    def momentum_magnitude(self):
        return np.linalg.norm(self._momentum)
    
    @property
    def velocity_magnitude(self):
        return np.linalg.norm(self._velocity)

    def update(self):
        self._momentum = self._mass*self._velocity
        self.energy = 0.5*self._mass*np.linalg.norm(self._velocity)**2

test_trader = Trader(1,2)

# Rewriting Simulation class


In [ ]:
class Simulation(mesa.Model):
    
    def __init__ (self,mean_collision_probability = 1, starting_velocity = 1,trader_mass = 2, no_traders = 100, sim_length=100, Dimensions = 2):
        self.no_traders = int(no_traders)
        self.simulation_length = int(sim_length)+1
        self.v0 = starting_velocity
        self.m = trader_mass
        self.dimensions = Dimensions
        self.mean_probability_of_collision = mean_collision_probability
        self.schedule = mesa.time.RandomActivation(self)


        
        self.init_traders()
        
    def init_traders(self):
        
        self.traders = []
        
        for i in range(self.no_traders):
            
            trader = Trader(velocities = np.full(self.dimensions, self.v0, dtype=float), m = 1, dimensions = self.dimensions)
            self.schedule.add(trader)
         
        self.datacollector = mesa.DataCollector(
            agent_reporters = {"Velocity": "velocity"}
        )

    
    def begin_simulation(self, collision_probability = 'all', collision_partner = 'random',collision_type = 'kinetic', collision_test = False, particle_history_test = False):
        self.type = collision_type
        self.collision_probability = collision_probability
        self.collision_partner = collision_partner
        self.collision_test = collision_test
        self.particle_history_test = particle_history_test
        
        for self.time in tqdm(range(1,self.simulation_length),desc='Colliding... '): 
            
            self.create_list_of_collisions()
            np.random.shuffle(self.list_of_collisions)
            
            self.determine_collision_partners()
            print(self.collision_partners)
            
            self.simulate_list_of_collisions()
        
        print('Simulation Complete!')
    
    def create_list_of_collisions(self):
        self.list_of_collisions = []
        
        if self.collision_probability == 'proportional':
            # loop through all particles
            for particle_index, particle in enumerate(self.particles):

                # Repeat collision check for velocities greater than 1
                number_of_checks = math.ceil(particle.velocity[0]) # Should I use magnitude velocity/net_assets
                check = 0
                collided = False
                while collided == False and check < number_of_checks:
                    collided = self.collision_check(particle)
                    check += 1

                if collided == True:
                    self.list_of_collisions.append(particle_index)
                    
                    
        elif self.collision_probability == 'all':
            # Maintain 50% chance of causing a collision
            particle_index_list = list(range(0,self.no_traders))
            np.random.shuffle(particle_index_list)
            self.list_of_collisions= particle_index_list[:int(len(particle_index_list)/2)]
            
    def collision_check(self, particle):
        
        # Chooses whether a particle will cause a collision
        # 50% chance to collide for 0.5
        # 25% chance to collide for 0.25
        
        if np.random.random() < self.mean_probability_of_collision:
            return True

    def determine_collision_partners(self):
        
        self.collision_partners = []

        # Creating list of collision partners
        if self.collision_partner == 'proportional':
            # Note that this does not prevent particles from colliding with themselves 
            # Equivalent to collision partner lottery with number of tickets bought being defined by wealth

            for i in range(self.no_traders):
                no_tickets = math.ceil(self.particles[i].net_assets)

                for j in range(no_tickets):
                    self.collision_partners.append(i)
                    
        elif self.collision_partner == 'random':
            
            self.collision_partners = list(range(self.no_traders))    
    
    
    def simulate_list_of_collisions(self):
        
        for collision_index, index1 in enumerate(self.list_of_collisions):
            
            index2 = random.choice(self.collision_partners)
            
            # Prevents particles from colliding with themselves
            if index1 == index2:
                self.update_parameters(index1,index2)
                continue
            
            if self.collision_test == True:
                v1before = self.particles[index1].velocity_magnitude
                v2before = self.particles[index2].velocity_magnitude

            self.collision(self.particles[index1],self.particles[index2])
            
            if self.collision_test == True:
                print(f'Particle {index1} velocity before {v1before}, after: {self.particles[index1].velocity_magnitude}, difference = {self.particles[index1].velocity_magnitude-v1before}')
                print(f'Particle {index2} velocity before {v2before}, after: {self.particles[index2].velocity_magnitude}, difference = {self.particles[index2].velocity_magnitude-v2before}')
            
            self.update_parameters(index1,index2)    
        
        # Updating parameters for those not involved in a collision
        for index in range(self.no_traders):
            self.update_parameters(index)

            
            
    def collision(self,particle1,particle2):

        # Select for type of collision
        if self.type == 'kinetic':
            self.kinetic_collision(particle1,particle2)
        elif self.type == 'exchange':
            self.exchange(particle1,particle2)
        elif self.type == 'transaction':
            self.transaction(particle1,particle2)
            
    def transaction(self,particle1,particle2):
        # Velocity is the equivalent to cash in this simulation
        # Here we will exchange velocity for commodities        
        if particle1.commodity_value > particle2.commodity_value and particle2.commodities > 0 and particle1.velocity[0] > particle1.commodity_value:
            particle1.commodities += 1
            particle1.velocity[0] -= particle1.commodity_value
            
            particle2.commodities -= 1
            particle2.velocity[0] += particle1.commodity_value
        elif particle2.commodity_value > particle1.commodity_value and particle1.commodities > 0 and particle2.velocity[0] > particle2.commodity_value:
            particle2.commodities += 1
            particle2.velocity[0] -= particle2.commodity_value
            
            particle1.commodities -= 1
            particle1.velocity[0] += particle2.commodity_value
        
        particle1.update()
        particle2.update()
        
    
    def exchange(self,particle1,particle2):
        
        self.kinetic_collision(particle1,particle2)
        
        total_commodities = particle1.commodities + particle2.commodities
        fractional_commodity_transfer = np.random.random()
        
        particle1.commodities = round(total_commodities*fractional_commodity_transfer)
        particle2.commodities = round(total_commodities*(1-fractional_commodity_transfer))
            
    def kinetic_collision(self,particle1,particle2):
        
        # Momentum is conserved
        # mv + m'v' (before) = mv + m'v' (after)

        p_before = np.ones(self.dimensions)
        p_after = np.ones((self.dimensions,2))
        v_after = np.ones((self.dimensions,2))
        fractional_momentum_transfer = np.ones(self.dimensions)

        for i in range(self.dimensions):
            p_before[i] = particle1.mass*particle1.velocity[i] + particle2.mass*particle2.velocity[i]
            
            # randomise transfer of energy
            fractional_momentum_transfer[i] = np.random.random()
        
            p_after[i,0] = fractional_momentum_transfer[i]*p_before[i]
            p_after[i,1] = fractional_momentum_transfer[i]*p_before[i]  
            
            v_after[i,0] = p_after[i,0]/particle1.mass
            v_after[i,1] = p_after[i,1]/particle1.mass
                     
            particle1.velocity[i] = v_after[i,0]
            particle2.velocity[i] = v_after[i,1]
            
            particle1.update()
            particle2.update()

    def update_parameters(self,*indices):
        
        for index in indices:
                    
            self.particle_history[0,index,self.time] = self.particles[index].mass
        
            self.particle_history[1,index,self.time] = self.particles[index].energy

            self.particle_history[2,index,self.time] = self.particles[index].velocity_magnitude

            self.particle_history[3,index,self.time] = self.particles[index].momentum_magnitude
        
            self.particle_history[4,index,self.time] = self.particles[index].commodities
        
            self.particle_history[5,index,self.time] = self.particles[index].commodity_value
        
            self.particle_history[6,index,self.time] = self.particles[index].net_assets
            
            for j in range(self.dimensions):
            
                if self.particle_history_test == True:
                    if self.particle_history[j+7,index,self.time] == 0.69:
                        vbefore = self.particle_history[j+7,index,self.time-1]
                    else:
                        vbefore = self.particle_history[j+7,index,self.time]

                self.particle_history[j+7,index,self.time] = self.particles[index].velocity[j]

                if self.particle_history_test == True:
                    diff = abs(self.particle_history[j+7,index,self.time] - vbefore)
                    print(f'Particle History @ Time {self.time}: particle {index} v Before = {vbefore}, After = {self.particle_history[j+7,index,self.time]}, Difference = {diff}')
                    if diff > 1:
                        print(f'Error: for particle {index} between velocity and previous velocity is greater than 1 @ time: {self.time}')

                self.particle_history[j+7+self.dimensions,index,self.time] = self.particles[index]._momentum[j]
            
    def fit_Maxwell_Boltzmann(self, bin_heights, bin_edges, ax, variable_being_fitted):
        bin_centres = bin_edges[:-1] + np.diff(bin_edges)/2
        ax.plot(bin_centres,bin_heights)

        opt_params, pcov = curve_fit(Maxwell_Boltzmann_2D_speed, bin_centres, bin_heights, p0=[1, 1, 1], bounds = (0,50))
        perr = perr = np.sqrt(np.diag(pcov))
        x_interval_for_fit = np.linspace(bin_edges[0], bin_edges[-1], 10000)
        fit_y = Maxwell_Boltzmann_2D_Simple(x_interval_for_fit, *opt_params)
        ax.plot(x_interval_for_fit, fit_y, label = 'f')

    def plot_data(self,time_steps_to_plot = [], MB_fit = False ,Mass = False, Energy = False, Velocity = True, Momentum = False, **other_variables):
        plot_data_arguments = {}
        self.time_steps_to_plot = time_steps_to_plot
        
        for key, value in locals().items():
            if type(value) == bool:
                plot_data_arguments[key] = value
        
        for key, value in other_variables.items():
            plot_data_arguments[key] = value
        
        size_of_plot = 0
        for key, value in plot_data_arguments.items():
            if value == True:size_of_plot += 1
        
        self.distributions = []
        
        plt.rcParams["figure.figsize"] = (20,size_of_plot*5)
        plt.rcParams["figure.dpi"] = 50
        plt.rcParams['font.size'] = 22
        
        fig = plt.figure(constrained_layout=True)        

        plot_index = 0

        variables_to_plot = []
        
        for i, (key,value) in enumerate(plot_data_arguments.items()):
            if value == True:
                variables_to_plot.append(key)

        plt.suptitle(f'Parameter Distributions at Different time step in {self.dimensions}-Dimension Collision Simulation',fontsize='xx-large')
        subfigs = fig.subfigures(nrows=size_of_plot, ncols=1)

        for row, subfig in enumerate(np.array(subfigs,ndmin=1)):
            subfig.suptitle(f'{variables_to_plot[row]} Distribution',fontsize='large')
            axs = subfig.subplots(nrows=1, ncols=len(time_steps_to_plot))
            for col, ax in enumerate(np.array(axs,ndmin=1)):
                bin_heights , bin_edges, _ = ax.hist(self.particle_history[self.particle_history_dict[variables_to_plot[row]],:,time_steps_to_plot[col]],bins=40,density = True,stacked = True)
                ax.set_title(f'Time = {time_steps_to_plot[col]}', fontsize = 'large')
                # ax.set_xlim(0,0.6*np.max(self.particle_history[self.particle_history_dict[variables_to_plot[row]],:,:]))
                if MB_fit == True:
                    self.fit_Maxwell_Boltzmann(bin_heights, bin_edges, ax, variables_to_plot[row])
                self.distributions.append(bin_heights)        
        

        plt.show()
        return self.distributions
    
# Manual simulation run if desired
sim = Simulation(no_traders=10,sim_length=10, Dimensions = 1)
sim.begin_simulation(collision_probability = 'proportional', collision_partner = 'random',collision_type = 'kinetic',collision_test= False, particle_history_test = False)